In [ ]:
# # For SRL parsing (dependencies outdated)
!pip install bertopic==0.14.1
!pip install --upgrade joblib==1.1.0
!pip install --upgrade tbb
!pip install stanza==1.4.0
!pip install jsonlines
!pip install jiwer==2.2.1
!pip install spacy[transformers]
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 48.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
    

In [ ]:
# list of stopwords
import nltk
nltk.download('brown')
nltk.download('universal_tagset')
from nltk.corpus import brown
all_words = []
tags = ['CCONJ','ADP','DET','INTJ','NUM','PRON','SCONJ']
for elem in brown.tagged_words(tagset="universal"):
  if elem[1] in tags:
    all_words.append(elem[0])

all_words = list(set(all_words))

import sklearn
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
stopwords = list(sklearn.feature_extraction.text.ENGLISH_STOP_WORDS)
stopwords.extend(all_words); print(len(stopwords))
vectorizer_model = CountVectorizer(stop_words=stopwords)


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


2274


In [ ]:
import os, sys
import torch
torch.manual_seed(0)

from transformers import set_seed
set_seed(42)

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import random
np.random.seed(0)
random.seed(0)

import nltk
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize

import ast
import itertools
from tqdm import tqdm
tqdm.pandas()
import itertools
import json



import re
# import stanza
# stanza.download('en')
# nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,ner,depparse',use_gpu=True)

import string
import jiwer

import gensim
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel


from transformers import BertTokenizer, AutoTokenizer, TFAutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from sentence_transformers import SentenceTransformer, CrossEncoder, util, models
cross_encoder = CrossEncoder('cross-encoder/stsb-distilroberta-base',device='cuda')
entail_encoder = CrossEncoder('cross-encoder/nli-distilroberta-base',device='cuda')

# sentence-transformers/all-mpnet-base-v2
word_embedding_model = SentenceTransformer('flax-sentence-embeddings/stackoverflow_mpnet-base',device='cuda')


import math
from scipy.spatial import distance
from scipy.signal import find_peaks
from scipy.stats import chi2_contingency
from scipy.spatial.distance import euclidean
from DBCV import DBCV

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.patches as mpatches
plt.rcParams["figure.figsize"] = (11.7,8.27)
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

main_arguments = ['ARG' + str(idx) for idx in range(6)]

from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
##load activities and sample for tuning bertopic
all_data = pd.read_csv('all_activities.csv');all_data.shape[0]
train_data = all_data.sample(100000,random_state=0)

all_rules = train_data['parsed_clauses'].tolist();print(len(all_rules))

100000


In [ ]:
##normalize terminology from 'podling' to project
def process_(text):
  text = text.replace("\r\n"," ").replace("\n"," ")
  text = re.sub('podling', 'project', text,flags=re.IGNORECASE)
  regex = r'\b(\w+)(?:\W+\1\b)+'
  text = re.sub(regex, r'\1', text, flags=re.IGNORECASE)
  return text.lower()

In [ ]:
##read ASF rules as clustering seeds

rules_data = pd.read_csv('asf_rules.csv')
rules = rules_data['rules'].tolist();print(len(rules))

422


In [ ]:

# grid search for best clustering
dbcv_min = 0

settings = [(100,100,100),(100,100,10),(10,100,100),(10,100,10)]

for samp,clus,neighbor in settings:
  for comp in [5,10,20]:
      umap_model = UMAP(n_neighbors=neighbor, n_components = comp, min_dist=0.0, metric='cosine', random_state=0)
      hdbscan_model = HDBSCAN(min_cluster_size=clus, metric='euclidean', min_samples = samp, cluster_selection_method = 'eom', prediction_data=True)
      topic_model = BERTopic(embedding_model=word_embedding_model,
                            seed_topic_list = rules,
                            top_n_words=3,
                            low_memory=True,
                            verbose=False,
                            umap_model=umap_model,
                            hdbscan_model = hdbscan_model,
                            calculate_probabilities=False,
                            vectorizer_model=vectorizer_model,
                            n_gram_range=(1,1))

      topic_model.hdbscan_model.gen_min_span_tree=True
      train_data['group_id'] = topic_model.fit_transform(all_rules)[0]

      freq = topic_model.get_topic_info()
      train_data['topic'] = train_data['group_id'].apply(lambda x : freq[freq['Topic'] == x].iloc[0]['Name'])


      topics = [[item for item in elem.split('_')[1:] if len(item) > 1] for elem in train_data['topic'].unique()]
      doc_tokenized = [[char for char in doc.split() if len(char) > 1] for doc in train_data['parsed_clauses'].tolist()];

      all_words = set(itertools.chain.from_iterable(doc_tokenized))
      final_topics = []

      for elem in topics:
        elem = list(set(elem).intersection(all_words))
        if elem : final_topics.append(elem)

      dictionary = corpora.Dictionary(doc_tokenized + final_topics)

      cm = CoherenceModel(topics=final_topics, texts = doc_tokenized, dictionary=dictionary, coherence='c_npmi',window_size=1000)
      cm2 = CoherenceModel(topics=final_topics, texts = doc_tokenized, dictionary=dictionary, coherence='c_v',window_size=1000)
      try:
        asf_tops,_ = topic_model.transform(rules)
        top_map = pd.DataFrame(columns=['rule','label'])
        top_map['rules'],top_map['label'] = rules,asf_tops
        top_map['topic'] = top_map['label'].apply(lambda x : freq[freq['Topic'] == x].iloc[0]['Name'])
        top_map.sort_values(by=['label'],inplace=True)
        top_map.to_csv('cluster_rules_outlier.csv',index=False)
        top_map = top_map[top_map['label'] > -1]
        print('Number of topics: ', len(top_map['label'].unique()))
        print('Hyperparameters :',samp,clus,neighbor,comp)
        print("cluster density:",topic_model.hdbscan_model.relative_validity_,'NPMI : ',cm.get_coherence(),'CV : ',cm2.get_coherence())
        if dbcv_min < topic_model.hdbscan_model.relative_validity_:
          freq = topic_model.get_topic_info(); freq.to_csv('topic_frequecncy.csv')
          dbcv_min = topic_model.hdbscan_model.relative_validity_; topic_model.save('topic_model_ase', save_embedding_model=False)
      except: continue

/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Number of topics:  26
Hyperparameters : 100 100 100 5
cluster density: 0.20221540719435876 NPMI :  0.18920633375667412 CV :  0.7132724821221385


/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Number of topics:  29
Hyperparameters : 100 100 100 10
cluster density: 0.24037755465781194 NPMI :  0.18762503385662826 CV :  0.7130368841810331


/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


...

Number of topics:  44
Hyperparameters : 10 100 10 5
cluster density: 0.2556550028528446 NPMI :  0.16003604111649344 CV :  0.6858639833218546


/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Number of topics:  42
Hyperparameters : 10 100 10 10
cluster density: 0.3219429460017643 NPMI :  0.15449263611722816 CV :  0.6835685099944763


/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Number of topics:  43
Hyperparameters : 10 100 10 20
cluster density: 0.2787980440616116 NPMI :  0.14544393218965992 CV :  0.675263620432288
